# 淘宝用户分析

## 项目背景

### 数据来源
https://tianchi.aliyun.com/dataset/dataDetail?dataId=649&userId=1

### 数据背景
UserBehavior是阿里巴巴提供的一个淘宝用户行为数据集，用于隐式反馈推荐问题的研究。

本数据集包含了2017年11月25日至2017年12月3日之间，有行为的约一百万随机用户的所有行为（行为包括点击、购买、加购、喜欢）。数据集的组织形式和MovieLens-20M类似，即数据集的每一行表示一条用户行为，由用户ID、商品ID、商品类目ID、行为类型和时间戳组成，并以逗号分隔。

### 数据概览
| 列名称     | 说明                                               |
| ---------- | -------------------------------------------------- |
| 用户ID     | 整数类型，序列化后的用户ID                         |
| 商品ID     | 整数类型，序列化后的商品ID                         |
| 商品类目ID | 整数类型，序列化后的商品所属类目ID                 |
| 行为类型   | 字符串，枚举类型，包括('pv', 'buy', 'cart', 'fav') |
| 时间戳     | 行为发生的时间戳                                   |

| 行为类型 | 说明                     |
| -------- | ------------------------ |
| pv       | 商品详情页pv，等价于点击 |
| buy      | 商品购买                 |
| cart     | 将商品加入购物车         |
| fav      | 收藏商品                 |

| 维度         | 数量        |
| ------------ | ----------- |
| 用户数量     | 987,994     |
| 商品数量     | 4,162,024   |
| 商品类目数量 | 9,439       |
| 所有行为数量 | 100,150,807 |

## 分析目标
1. 当日PV、当日UV、日均PV、活跃用户数、交易用户数、活跃用户比例、交易用户比例、用户活跃天数
1. 用户活跃时期、时段
2. 用户对哪些产品、类目感兴趣
3. 用户、行为转化率
4. 根据RFM模型对用户分类

## 数据准备

### 导入数据

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pyecharts import charts as pyc
from pyecharts import options as opts
import warnings
%matplotlib inline
# 忽略警告
warnings.filterwarnings("ignore")
# 解决matplotlib中文乱码
plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False
# 作图的字体默认设置
fontdict = {'fontsize': 15,
            'horizontalalignment': 'center'}

In [ ]:
# 数据量过大，这里只选取前1000w行
# 根据官网介绍给定列名
columns = ["userid", 'itemid', 'categoryid', 'type', 'timestamp']
# 使用iterator将数据分块，返回迭代器
data = pd.read_csv("./UserBehavior.csv", iterator=True,
                   chunksize=10000000, names=columns)

df = data.get_chunk(10000000)

In [ ]:
# 数据的基本信息，共1000万条，前三个字段是int，后两个是object
df.info()

In [ ]:
# 查看数据前几行信息
df.head()

### 数据清洗
1. 将时间戳转为日期
2. 数据集说明中写的是本数据集的日期范围是2017年11月25日至2017年12月3日，所以剔除这日期以外的数据

In [ ]:
import time


def get_unixtime(timeStamp):
    formatStr = "%Y-%m-%d %H:%M:%S"
    tmObject = time.strptime(timeStamp, formatStr)
    return int(time.mktime(tmObject))


startTime = get_unixtime("2017-11-25 00:00:00")
endTime = get_unixtime("2017-12-03 23:59:59")

# 保留2017年11月25日至2017年12月3日之间的数据
df = df[(df.timestamp >= startTime) & (df.timestamp <= endTime)]
# 要注意时区问题，pd.to_datetime转换的默认时区不是中国
df['datetime'] = pd.to_datetime(
    df.timestamp, unit='s', utc=True).dt.tz_convert("Asia/Shanghai")
# 删除时间戳节约内存
df.drop('timestamp', inplace=True, axis=1)
# 将datetime进行拆分成date和hour
df['date'] = df.datetime.dt.date
df['hour'] = df.datetime.dt.hour

## 数据分析

### 用户流量类指标
1. 2017-11-25至2017-12-03这9天内，用户操作累计浏览详情页8944500次，累计添加购物车559127次，累计收藏产品291657次，累计购买199143次
2. 平均每日浏览量993833次，添加购物车62125次，收藏产品32406次，购买22127次
3. 总计97,810名用户在这段时间内使用过淘宝app，平均每人浏览91次，添加购物车6次，收藏产品3次，购买2次
4. 用户跳失率为5.74%（只看不买的用户占比）,复购率为66.01%
5. 相较于浏览->收藏->购买的操作流程，用户更喜欢浏览->加入购物车->购买这种方式，前者的用户转化率为9%而后者为23%
6. 从整体用户行为来看89%的操作是浏览商品，只有2%的操作是购买，行为转化率过低
7. 付费用户相较于非付费用户人均多贡献了35.84%的PV，也就是说付费用户在淘宝app上的使用时间高于非付费用户，应该建立深度的用户画像，进行商品推荐。

In [ ]:
# 取消科学计数法，保留两位小数
pd.set_option("float_format", lambda x: "%.2f" % x)

# 9天总用户量,有操作记录的商品及类目
count_users = df.userid.nunique()
count_itemid = df.itemid.nunique()
count_categoryid = df.categoryid.nunique()
count_user_bought = df[df['type'] == 'buy'].userid.nunique()
count_user_nobought = count_users-count_user_bought

# 9天总操作记录数
types = ["pv", "cart", "fav", "buy"]
type_s = df.type.value_counts()

# 拼接9天所有、日均、人均指标
type_df = pd.DataFrame([type_s, type_s/9, type_s / count_users],
                       columns=types, index=["all", "aver_day", "aver_user"])

# 购买过的用户总共浏览了几次,购买了几次
type_df.loc['user_bought'] = df[df['userid'].isin(
    df[df['type'] == 'buy']['userid'].unique())].type.value_counts()

# 没有购买过的用户总共浏览了几次购买了几次
type_df.loc['user_nobought'] = type_df.loc['all']-type_df.loc['user_bought']

In [ ]:
type_df

In [ ]:
t1 = type_df.copy()
t1.index=['总计','平均每日','平均每用户','付费用户','非付费用户']
t1.columns=['浏览量','加入购物车','收藏','购买']
t1

In [ ]:
print(f"UV：{count_users}")
print(f"付费用户数：{count_user_bought}")
print(f"付费用户占比：{count_user_bought/count_users*100:.2f}%")
print(f"商品数：{count_itemid}")
print(f"类目数：{count_categoryid}")
# 非付费用户是否比付费用户贡献了更多的PV？
print(f"付费用户人均贡献PV：{type_df.loc['user_bought'].pv/count_user_bought:.2f}")
print(f"非付费用户人均贡献PV：{type_df.loc['user_nobought'].pv/count_user_nobought:.2f}")
print(
    f"付费用户相较于非付费用户人均多贡献了{((type_df.loc['user_bought'].pv/count_user_bought)/(type_df.loc['user_nobought'].pv/count_user_nobought)-1)*100:.2f}%的PV")

跳失率=只有点击行为的用户/总用户数
> 其实真正的跳失率应该是只浏览一个页面就离开的访问次数 / 该页面的全部访问次数，这边只是为了突出这些有待发展的客户

复购率=购买2次及以上用户数/总购买用户数
> 复购率可以分为按客户计算和按交易计算，这里我采用的是按客户计算。
> 一定要确定统计周期，像这个数据的统计周期就是9天

In [ ]:
# 将数据按照userid进行分组
groupby_userid = df.groupby(by=df.userid)
# unstack()可以展开再成为一个dataframe
user_type = groupby_userid.type.value_counts().unstack()

# 跳失率
# user_type.sum(axis=1)对DataFrame进行横向相加，如果一个userid的pv值==横向相加的和，那就表明他只有点击行为，没有购买/加入购物车/收藏的行为
only_pv_users = user_type[user_type['pv'] == user_type.sum(axis=1)]
# shape返回的是元组（行数，列数），所以用[0]取出行数
bounce_rate = only_pv_users.shape[0]/count_users

# 复购率
user_bought_twice = user_type[user_type['buy'] >= 2].shape[0]
user_bought = user_type[user_type['buy'] >= 1].shape[0]
repurchase_rate = user_bought_twice/user_bought

print("跳失率:{:.2f}%".format(bounce_rate*100))
print("复购率:{:.2f}%".format(repurchase_rate*100))

In [ ]:
user_type

用户转化率\
因为收藏和加入购物车没有必然关系，所以我们可以将用户的行为路径划分为两种来进行分析\
- process1:浏览->收藏->购买
- process2:浏览->加入购物车->购买
> 因为每种type都有两种可能性，共有4种type，所以其实总共有16种路径

In [ ]:
# 分别将fav和buy的df提取出来
pv_df = df[df['type'] == 'pv']
fav_df = df[df['type'] == 'fav']
buy_df = df[df['type'] == 'buy']
cart_df = df[df['type'] == 'cart']
# 将fav_df和buy_df合并,得到收藏后购买的用户人数
# process1 浏览->收藏->购买
pv_fav_df = pd.merge(left=pv_df, right=fav_df, how='inner', on=[
    'userid', 'itemid', 'categoryid'], suffixes=('_pv', '_fav'))
fav_buy_df = pd.merge(left=fav_df, right=buy_df, how='inner', on=[
                      'userid', 'itemid', 'categoryid'], suffixes=('_fav', '_buy'))
count_user_pv_fav = pv_fav_df[pv_fav_df.datetime_pv <
                              pv_fav_df.datetime_fav].userid.nunique()
count_user_fav_buy = fav_buy_df[fav_buy_df.datetime_fav <
                                fav_buy_df.datetime_buy].userid.nunique()
# process2 浏览->添加购物车->购买
pv_cart_df = pd.merge(left=pv_df, right=cart_df, how='inner', on=[
                      'userid', 'itemid', 'categoryid'], suffixes=('_pv', '_cart'))
cart_buy_df = pd.merge(left=cart_df, right=buy_df, how='inner', on=[
                       'userid', 'itemid', 'categoryid'], suffixes=('_cart', '_buy'))
count_user_pv_cart = pv_cart_df[pv_cart_df.datetime_pv <
                                pv_cart_df.datetime_cart].userid.nunique()
count_user_cart_buy = cart_buy_df[cart_buy_df.datetime_cart <
                                  cart_buy_df.datetime_buy].userid.nunique()

# 将数据合并成元组的列表是为了后面用pyecharts画图，如果不画图可以不这样合并
process1 = [("浏览人数", count_users), ("收藏人数", count_user_pv_fav),
            ("购买人数", count_user_fav_buy)]
process2 = [("浏览人数", count_users), ("添加购物车人数", count_user_pv_cart),
            ("购买人数", count_user_cart_buy)]

conversion = pd.DataFrame(data=[[user[1] for user in process1], [user[1] for user in process2]], index=[
                          'process1', 'process2'], columns=['pv', 'fav/cart', 'buy'])
conversion['pv_fav/cart'] = conversion['fav/cart'].div(conversion['pv'])
conversion['fav/cart_buy'] = conversion['buy'].div(conversion['fav/cart'])
conversion['pv_buy'] = conversion['buy'].div(conversion['pv'])

In [ ]:
# 用户转化率
conversion

In [ ]:
# 行为转化率

type_s/type_s.sum()

process1的用户转化率为9%，而process2的用户转化率为23%
 
process1：18%的用户会在浏览后收藏产品，其中只有47%的用户会购买该商品

process2：41%的用户会在浏览后把商品添加到购物车，其中有55%的用户会购买该商品

无论是从柱状图的高矮、漏斗图的粗细和数据比较我们都可以发现，process2的转化率更高，相较于收藏商品，用户更喜欢在购买商品前将商品放入购物车

- 我们从淘宝app的设计来看一下，进入淘宝app后最底下一行很明显就有购物车的选择图标，而要进入收藏必须通过“我的淘宝->收藏夹”进行查看，多一步操作就多一种客户流失的可能性。
- 再从两个界面进行考虑，加入购物车的商品很大概率会是用户已经选择好参数的商品（例如衣服尺寸、食品口味、3C产品配置等），点开购物车用户即可看到当初已经选择好的商品的现价（双11前夕我相信剁手党不可能只把商品收藏而不提前加入购物车吧）；而收藏页面显示的只是商品概览（图片、标题、价格等），并没有用户自定义的参数选择，如若客户从收藏页进入再购买，势必又要多几步操作。

为什么用户更喜欢在购买前先将商品放入购物车，推测原因：
1. 同款产品不同店铺比价
2. 凑单满减
3. 先放入购物车，等有优惠活动再购买

所以可以在商品详情页、商品图品等醒目地方添加提示用户将商品加入购物车的标语，或者添加限时优惠券，让顾客产生购买的紧迫感，刺激消费可能。

行为转化率\
89%的操作都是在看，而只有2%的操作进行了购买

In [ ]:
# 行为转化率
pyc.Bar().add_xaxis(['pv', 'fav/cart', 'buy']).add_yaxis(series_name="process1", yaxis_data=[int(x) for x in conversion.loc['process1'].values]).add_yaxis(
    series_name="process2", yaxis_data=[int(x) for x in conversion.loc['process2'].values]).set_series_opts(label_opts=opts.LabelOpts(is_show=False)).set_global_opts(
    title_opts=opts.TitleOpts(
        title="用户行为转化率"),
    toolbox_opts=opts.ToolboxOpts(is_show=True)
).render_notebook()

In [ ]:
# 这里就不重复作图了，用一个条形图就可以明显比较转化路径
# process1:浏览->收藏->购买
# pyc.Funnel(init_opts=opts.InitOpts(width='500px', height='500px')).add("type", process1, label_opts=opts.LabelOpts(position="top"), tooltip_opts=opts.TooltipOpts(is_show=True), gap=5).set_global_opts(title_opts=opts.TitleOpts(title="操作转化率", subtitle="process1 浏览->收藏->购买")).render_notebook()
# process2:浏览->加入购物车->购买
# pyc.Funnel(init_opts=opts.InitOpts(width='500px', height='500px')).add("type", process2, label_opts=opts.LabelOpts(position="top"), tooltip_opts=opts.TooltipOpts(is_show=True), gap=5).set_global_opts(title_opts=opts.TitleOpts(title="操作转化率", subtitle="process2 浏览->添加购物车->购买")).render_notebook()

In [ ]:
# 用户加入购物车后/收藏产品后多久会购买
buy_after_cart = pd.DataFrame(
    cart_buy_df.datetime_buy-cart_buy_df.datetime_cart, columns=['interval'])
buy_after_fav = pd.DataFrame(
    fav_buy_df.datetime_buy-fav_buy_df.datetime_fav, columns=['interval'])
# 将timedelta格式转换为小时
buy_after_cart['hours'] = buy_after_cart['interval'].map(
    lambda x: x.total_seconds()//3600)
buy_after_fav['hours'] = buy_after_fav['interval'].map(
    lambda x: x.total_seconds()//3600)
# 有些用户会先购买，再收藏/加入购物车，再次购买(按照数据来看真的会有这种用户)，所以我们剔除这种先购买再收藏/加入购物车的操作记录
# 即删除负值
buy_after_cart = buy_after_cart[buy_after_cart['hours'] >= 0]
buy_after_fav = buy_after_fav[buy_after_fav['hours'] >= 0]

# 因为会出现大量的0小时，就是购买行为和添加购物车/收藏行为不满一小时
# 所以我们加个1，这样理解起来就是两种操作是间隔一小时
buy_after_cart['hours'] = buy_after_cart['hours']+1
buy_after_fav['hours'] = buy_after_fav['hours']+1

In [ ]:
buy_after_cart.hours.describe()

In [ ]:
buy_after_fav.hours.describe()

In [ ]:
buy_after_cart.hours.value_counts(1).sort_index()

In [ ]:
buy_after_fav.hours.value_counts(1).sort_index()

In [ ]:
def hour_cat(s):
    # 将间隔超过48小时的数据都修改为49
    # 自定义间隔时长将数据进行分区（0-1小时设置为1，1-2小时设置为2...以此类推）
    bins = [0.0, 1.0, 2.0, 3.0, 6.0, 9.0, 12.0, 24.0, 48.0, np.inf]
    labels = [1, 2, 3, 6, 9, 12, 24, 48, 49]
    s.where(s <= 48, 49)
    s = pd.cut(s, bins=bins, labels=labels)
    # 将分类统计百分比，并按照索引升序排序
    return s.value_counts(1).sort_index()


buy_after_cart_cat = hour_cat(buy_after_cart['hours'])
buy_after_fav_cat = hour_cat(buy_after_fav['hours'])

In [ ]:
# 用户不同操作路径的购买间隔
# 经过分层的直方图
fig = plt.figure(figsize=(15, 6))

ax1 = plt.subplot(1, 2, 1)
sns.barplot(x=buy_after_cart_cat.index,
            y=buy_after_cart_cat.values, color='#3274a1')
ax1.set_xlabel("用户加入购物车后购买间隔(小时)", fontdict=fontdict)


ax2 = plt.subplot(1, 2, 2)
sns.barplot(x=buy_after_fav_cat.index,
            y=buy_after_fav_cat.values, color='#ffdb99')
ax2.set_xlabel("用户收藏后购买间隔(小时)", fontdict=fontdict)
print("用户不同操作路径的购买间隔图(分组)")
plt.savefig("./1.png")
plt.show()


In [ ]:
# 用户不同操作路径的购买间隔
# 这是没有经过分层的直方图
plt.figure(figsize=(20, 6))
fontdict = {'fontsize': 15,
            'horizontalalignment': 'center'}

ax1 = plt.subplot(1, 2, 1)
ax1.hist(buy_after_cart.hours, bins=30, color='#a5c8e1')
ax1.set_xlabel("用户加入购物车后购买间隔(小时)", fontdict=fontdict)

ax2 = plt.subplot(1, 2, 2)
ax2.hist(buy_after_fav.hours, bins=30, color='#ffdb99')
ax2.set_xlabel("用户收藏后购买间隔(小时)", fontdict=fontdict)

print("用户不同操作路径的购买间隔图(未分组)")
plt.show()

- 无论是哪一种路径，用户在1小时之内购买的可能性都较大，其中收藏后购买的用户中有45%的用户在收藏后1小时内就下单购买，而加入购物车后再购买的时间间隔明显较长，添加购物车后购买的用户中有超过60%的用户在加入购物车12小时之后再进行购买。
- 如有用户收藏或加入购物车后可有立即的客服跟进，例如优惠放送、话术吸引，则则很可能提高用户快速购买的可能性，进一步提高购买转化率。

### 每日情况
1. 浏览量、收藏数、添加购物车次数、购买次数、用户量在2017/12/02当天急速上升。
2. 周末的各项指标都优于周内，周六周日用户尤为活跃，用户有更多的时间可以逛淘宝，所以应当在周六周日当天进行适当的营销活动。
3. 每日的活跃用户量基本在70%上下浮动，而成交客户基本维持在20%。
3. 这9天内，用户活跃天数呈现正态分布，用户活跃天数主要集中在3-7天（用户当天操作记录达3条算作当天活跃）

In [ ]:
# 首先按照date进行分组
import datetime
groupby_date = df.groupby(df.date)

# 将日期字符串先转为日期格式
days = pd.to_datetime(['2017/11/25', '2017/11/26', '2017/11/27', '2017/11/28',
                       '2017/11/29', '2017/11/30', '2017/12/01', '2017/12/02', '2017/12/03'])

# 将日期作为索引
daily_df = pd.DataFrame(data=None, columns=types, index=days)

# 通过循环获得每组的统计数据
for day in days:
    daily_df.loc[day] = groupby_date.get_group(day).type.value_counts()

# 我们认为当天使用淘宝app进行3次操作（无论是查看详情页、收藏、加入购物车还是购买都算）的用户就是活跃用户
# 自定义活跃用户操作次数基准
active_user_standard = 3
# 添加每日UV
daily_df['uv'] = [groupby_date.get_group(
    day).userid.nunique() for day in daily_df.index]
# 根据date和userid分组后计算当天操作次数达到基准或以上的用户数作为活跃用户数
daily_df['dau'] = [(groupby_date.get_group(day).groupby(by='userid').size(
) > active_user_standard).value_counts()[True] for day in daily_df.index]
# 活跃用户比例
daily_df['au_rate'] = daily_df['dau']/daily_df['uv']
# 交易用户数
daily_df['buyer'] = df[df['type'] == 'buy'][['date', 'userid']
                                            ].groupby(by=['date', 'userid']).size().count(level=0)
# 交易用户比例
daily_df['buyer_rate'] = daily_df['buyer']/daily_df['uv']
# 判断当天是周几
daily_df['weekday'] = [datetime.datetime.isoweekday(
    datetime.date(x.year, x.month, x.day)) for x in daily_df.index]
# 用户活跃天数
temp = df[['userid', 'date', 'type']].groupby(by=['date', 'userid']).count()
user_active_days_df = temp[temp['type'] > active_user_standard].count(level=1)
sorted_user_active_df = user_active_days_df['type'].value_counts().sort_index()

In [ ]:
daily_df

In [ ]:
# 用户行为变化趋势图
pyc.Line(init_opts=opts.InitOpts(
    width='800px', height='600px')).add_xaxis(xaxis_data=daily_df.index.date).add_yaxis(
    series_name='浏览量',
    y_axis=daily_df.pv, is_selected=True, is_symbol_show=False, is_smooth=False
).add_yaxis(
    series_name='加入购物车',
    y_axis=daily_df.cart, is_selected=True, is_symbol_show=False, is_smooth=False
).add_yaxis(
    series_name='收藏商品',
    y_axis=daily_df.fav, is_selected=True, is_symbol_show=False, is_smooth=False
).add_yaxis(
    series_name='购买商品',
    y_axis=daily_df.buy, is_selected=True, is_symbol_show=False, is_smooth=False
).set_global_opts(
    title_opts=opts.TitleOpts(
        title="用户行为变化趋势图", subtitle="2017/11/25-2017/12/03期间用户行为变化趋势"),
    tooltip_opts=opts.TooltipOpts(trigger='axis'),
    toolbox_opts=opts.ToolboxOpts(is_show=True),
    xaxis_opts=opts.AxisOpts(type_='category', boundary_gap=False)
).render_notebook()

In [ ]:
# 用户量变化趋势图
pyc.Line(init_opts=opts.InitOpts(
    width='800px', height='600px')).add_xaxis(xaxis_data=daily_df.index.date).add_yaxis(
    series_name='当日用户数',
    y_axis=daily_df.uv, is_selected=True, is_symbol_show=False, is_smooth=False
).add_yaxis(
    series_name='活跃用户数',
    y_axis=daily_df.dau, is_selected=True, is_symbol_show=False, is_smooth=False
).add_yaxis(
    series_name='交易用户数',
    y_axis=daily_df.buyer, is_selected=True, is_symbol_show=False, is_smooth=False
).set_global_opts(
    title_opts=opts.TitleOpts(
        title="用户量变化趋势图", subtitle="2017/11/25-2017/12/03期间用户量变化趋势"),
    tooltip_opts=opts.TooltipOpts(trigger='axis'),
    toolbox_opts=opts.ToolboxOpts(is_show=True),
    xaxis_opts=opts.AxisOpts(type_='category', boundary_gap=False)
).render_notebook()

In [ ]:
# 用户量变化趋势图
pyc.Line(init_opts=opts.InitOpts(
    width='800px', height='600px')).add_xaxis(xaxis_data=daily_df.index.date).add_yaxis(
    series_name='当日用户数',
    y_axis=daily_df.uv, is_selected=True, is_symbol_show=False, is_smooth=False
).add_yaxis(
    series_name='活跃用户数',
    y_axis=daily_df.dau, is_selected=True, is_symbol_show=False, is_smooth=False
).add_yaxis(
    series_name='交易用户数',
    y_axis=daily_df.buyer, is_selected=True, is_symbol_show=False, is_smooth=False
).set_global_opts(
    title_opts=opts.TitleOpts(
        title="用户量变化趋势图", subtitle="2017/11/25-2017/12/03期间用户量变化趋势"),
    tooltip_opts=opts.TooltipOpts(trigger='axis'),
    toolbox_opts=opts.ToolboxOpts(is_show=True),
    xaxis_opts=opts.AxisOpts(type_='category', boundary_gap=False)
).render("./2.html")

In [ ]:
# 用户累计活跃天数分布图
pyc.Bar(init_opts=opts.InitOpts(width='800px', height='600px')).add_xaxis(xaxis_data=[int(x) for x in sorted_user_active_df.index]).add_yaxis(series_name='用户活跃天数', yaxis_data=[int(
    x) for x in sorted_user_active_df.values]).set_series_opts(label_opts=opts.LabelOpts(is_show=False)).set_global_opts(title_opts=opts.TitleOpts(title="用户累计活跃天数分布图"), toolbox_opts=opts.ToolboxOpts(is_show=True)).render_notebook()

In [ ]:
# 用户累计活跃天数分布图
pyc.Bar(init_opts=opts.InitOpts(width='800px', height='600px')).add_xaxis(xaxis_data=[int(x) for x in sorted_user_active_df.index]).add_yaxis(series_name='用户活跃天数', yaxis_data=[int(
    x) for x in sorted_user_active_df.values]).set_series_opts(label_opts=opts.LabelOpts(is_show=False)).set_global_opts(title_opts=opts.TitleOpts(title="用户累计活跃天数分布图"), toolbox_opts=opts.ToolboxOpts(is_show=True)).render("./3.html")

### 时段情况
1. 各时段指标走势大致一致。
2. 5:00-10:00及18:00-21:00两段时间是用户活跃度迅速增长的时段。
3. 10:00-18:00这个时间段的用户活跃度上下浮动，但基本保持着同一水平。
4. 13:00，15:00这两个时间段是用户活跃度稳定阶段的两个小高峰，我们可以猜测13:00是因为用户工作午休结束了拿起手机再摸一会儿鱼造成的，15:00是因为工作了一两个小时又想摸鱼。17:00开始一部分用户下班了或马上下班，可以开始正大光明摸鱼。
5. 21:00达到用户活跃度峰值。

In [ ]:
# 根据时段进行分组
groupby_hour = df.groupby(by=df.hour)

# 创建一个空的时段dataframe
# 列表生成式创建一个0-23的时间列表
times = [x for x in range(0, 24)]
time_df = pd.DataFrame(data=None, index=times, columns=types)

for t in times:
    time_df.loc[t] = groupby_hour.get_group(t).type.value_counts()
time_df['uv'] = [groupby_hour.get_group(
    hour).userid.nunique() for hour in times]

# 交易用户活跃时段
groupby_hour_userid = df[df['type'] == 'buy'][[
    'userid', 'hour']].groupby(by=['hour', 'userid'])
buyer_hour = groupby_hour_userid.size().count(level=0)

In [ ]:
time_df

In [ ]:
# 绘制折线图
pyc.Line(init_opts=opts.InitOpts(
    width='800px', height='600px')).add_xaxis(xaxis_data=time_df.index).add_yaxis(
    series_name='pv',
    y_axis=time_df.pv, is_selected=True, is_symbol_show=False,
).add_yaxis(
    series_name='cart',
    y_axis=time_df.cart, is_selected=True, is_symbol_show=False
).add_yaxis(
    series_name='fav',
    y_axis=time_df.fav, is_selected=True, is_symbol_show=False
).add_yaxis(
    series_name='buy',
    y_axis=time_df.buy, is_selected=True, is_symbol_show=False
).set_global_opts(
        title_opts=opts.TitleOpts(title="用户行为时段分布图", subtitle="用户行为的时段分布情况"),
        tooltip_opts=opts.TooltipOpts(trigger="axis"),
        toolbox_opts=opts.ToolboxOpts(is_show=True),
        xaxis_opts=opts.AxisOpts(type_="category", boundary_gap=False),
).render_notebook()

In [ ]:
# 绘制折线图
pyc.Line(init_opts=opts.InitOpts(
    width='800px', height='600px')).add_xaxis(xaxis_data=time_df.index).add_yaxis(
    series_name='用户量',
    y_axis=time_df.uv, is_selected=True, is_symbol_show=False,
).add_yaxis(
    series_name='交易用户',
    y_axis=[int(x) for x in buyer_hour.values], is_selected=True, is_symbol_show=False,
).set_global_opts(
        title_opts=opts.TitleOpts(title="用户活跃时段", subtitle="用户活跃度的时段分布情况"),
        tooltip_opts=opts.TooltipOpts(trigger="axis"),
        toolbox_opts=opts.ToolboxOpts(is_show=True),
        xaxis_opts=opts.AxisOpts(type_="category", boundary_gap=False),
).render_notebook()

### 产品/类目情况
1. itemid为3122135、3031354等一些商品在这9天内属于热销产品，总订单量都在50单以上。
2. 1464116、2735466等类目的是这9天内销售量较高的类目。

In [ ]:
# 按照itemid进行分组
groupby_itemid = df.groupby(by='itemid')
item_type_df = groupby_itemid.type.value_counts().unstack()
item_type_df.replace(to_replace=np.nan, value=0, inplace=True)
# 按照categroyid进行分组
groupby_categoryid = df.groupby(by='categoryid')
cat_type_df = groupby_categoryid.type.value_counts().unstack()
# cat_type_df.replace(to_replace=np.nan,value=0,inplace=True)

In [ ]:
# 热卖商品top10
item_type_df.sort_values(by='buy', ascending=False)[:10]

In [ ]:
# 热销类目top10
cat_type_df.sort_values(by='buy', ascending=False)[:10]

In [ ]:
item_type_df

In [ ]:
# 看看非付费用户比较爱看哪些产品
groupby_itemid_userid = df[~df['userid'].isin(df[df['type'] == 'buy'].userid.unique())][[
    'userid', 'itemid', 'type']].groupby(by=['itemid', 'userid'])

groupby_itemid_userid.size().count(level=0).sort_values(ascending=False)[:20]

对于这些用户经常看而不买的商品，我们可以通过优惠券、红包、返利或是推送更具有性价比的同类商品给用户，促进用户购买

In [ ]:
# 一件商品在被购买前会经过多少次操作(多少次操作才能换来一次购买)
item_type_df['behavior_transform'] = [
    round(x, 0) for x in item_type_df.sum(axis=1)/item_type_df.buy]

In [ ]:
# 绘制散点图看商品类别
# inf类型在这个图不报错
plt.figure(figsize=(10, 8))
sns.scatterplot(x=item_type_df.behavior_transform,
                y=item_type_df.buy, alpha=0.3)

plt.xlim(0, 400)
plt.xlabel("平均操作次数(行为数/购买数)", fontdict=fontdict)
plt.ylabel("购买次数", fontdict=fontdict)
plt.title("产品类别划分", fontdict=fontdict)
plt.savefig("./11.png")
plt.show()

- 左上角属于快消/高频/品牌选择少/寡头
- 左下角属于低频/贵重/品牌选择少/寡头
- 右上角属于快消/高频/品牌选择多
- 右下角属于决策慎重/低频/品牌选择多

大部分购买行为发生在累计用户行为0-100次，用户喜欢购买的产品分类是属于左下角，要让分布逐渐往右上靠，从平台方考虑可以增加商品大类的专区假设，减少用户搜索对比，提升用户体验。若想让分布先往左上角移动可以考虑建设高频网红专区，或者在用户搜索时首先展示用户购买过或收藏过的品牌/产品。

In [ ]:
# 剔除np.inf查看购买前操作次数
behavior_times = item_type_df.behavior_transform.value_counts()
behavior_times.drop(np.inf, inplace=True)
behavior_times.sort_index(inplace=True)

In [ ]:
pyc.Bar().add_xaxis(
    xaxis_data=[int(x) for x in behavior_times.index]).add_yaxis(
        series_name='',
        yaxis_data=[int(x) for x in behavior_times.values]).set_series_opts(
            label_opts=opts.LabelOpts(is_show=False)).set_global_opts(
                title_opts=opts.TitleOpts(title="购买前平均操作次数分布"),
                toolbox_opts=opts.ToolboxOpts(is_show=True)).render_notebook()

两次操作就达成购买行为的占比很高（浏览一次就买），大多数购买前累计操作次数分布在0-100次，累计100次以上操作才换来一次购买行为的产品很多，如果有更详细的数据，例如在网页中埋点记录详情页留存率及用户浏览时长，就可以判断是在哪一阶段造成的用户流失或跳失率高的原因。

用户操作多，但是就不买，无外乎价格、质量、评价、产品介绍等方面，虽然说数据不可以成为“皇帝的新衣”，但是让用户产生购买“皇帝新衣”的冲动还是可以考虑的。

### RFM模型
RFM模型是衡量客户价值和客户创利能力的重要工具和手段。在众多的客户关系管理(CRM)的分析模式中，RFM模型是被广泛提到的。该机械模型通过一个客户的近期购买行为、购买的总体频率以及花了多少钱3项指标来描述该客户的价值状况。
- 重要价值客户最近消费时间近，消费频次和消费金额高，必须重点关注并保持住，需要提高用户满意度以及粘性
- 重要保持客户最近消费时间远，但消费频次和消费金额高，可以认为是有段日子没有来的忠诚客户，要主动与其联系，关注社群运营
- 重要发展客户最近消费时间近，消费金额高，频次低，用户忠诚度不高，但很可能是潜力客户，需要着重发展
- 重要挽留客户最近消费时间远，消费金额高，频次低，很可能成为流失用户，应该采取挽留措施

在这就不多具体介绍这个模型了，因为数据集缺少交易额，所以这里就只能实现RF模型


In [ ]:
# R:Recency最近一次交易日期；F:Frequency频率，交易次数
buy_userid_group = df[df['type'] == 'buy'].groupby(by='userid')

# 先创建一个空的DataFrame
RF = pd.DataFrame(index=buy_userid_group.groups.keys(), columns=['R', 'F'])

# 因为这边索引的顺序就是按照groupby之后的顺序，所以直接赋值即可
RF['F'] = buy_userid_group.type.value_counts().values
RF['last_buy_time'] = buy_userid_group.datetime.max().values

# 我们假设获取数据集的第二天进行分析，所以选择2017/12/04为对照日期，保留天数
RF['R'] = (pd.to_datetime('2017-12-04')-RF['last_buy_time']).dt.days

In [ ]:
# 最近的一次交易记录距离2017/12/04为0天
# 即12/03购买相差还不到一天，最远的为9天，平均为2.6天
# R我们划分4个区域 0-1,2-3,4-6,7-9,分别得分4，3，2，1
RF.R.describe()

In [ ]:
# 交易次数最少为1次最多为93次,平均3次
# 这里的3次分母是所有购买过商品的用户
# 而之前得出的人均购买次数2次分母是所有用户
# F划分4个区域 1,2,3,4+，分别得分1，2，3，4
RF.F.describe()

In [ ]:
def R_score(x):
    if 0 <= x <= 1:
        return 4
    elif 2 <= x <= 3:
        return 3
    elif 4 <= x <= 6:
        return 2
    elif 7 <= x <= 9:
        return 1
    else:
        return 0


def F_score(x):
    if x == 1:
        return 1
    elif x == 2:
        return 2
    elif x == 3:
        return 3
    elif x >= 4:
        return 4
    else:
        return 0


# 根据R,F进行评分
RF['R_score'] = RF.R.map(R_score)
RF['F_score'] = RF.F.map(F_score)

In [ ]:
# F_score平均为2.36，我们取2为中间值来衡量
# 这里的F_score和分类模型评价指标的F-score不是一个东西
RF.F_score.describe()

In [ ]:
# R评分平均为3.02，我们取3为中间值来衡量
RF.R_score.describe()

In [ ]:
def user_classfication(tup):
    R_score, F_score = tup
    if R_score >= 3 and F_score > 2:
        return "重要保持客户"
    elif R_score >= 3 and F_score <= 2:
        return "重要发展客户"
    elif R_score < 3 and F_score > 2:
        return "重要价值客户"
    elif R_score < 3 and F_score <= 2:
        return "重要挽留客户"
    else:
        return None


RF['user_classification'] = RF[['R_score', 'F_score']].apply(
    user_classfication, axis=1)

In [ ]:
RF

In [ ]:
# 统计用户分类情况
RF.user_classification.value_counts()

In [ ]:
# 用户分类占比
RF.user_classification.value_counts(1)

## 总结
1. 12/02-12/03相较于其他日期用户活跃度、各指标增长明显，与同样是周末的11/25-11/26形成较大差距，猜测是该周末有运营促销活动相关（应该是临近双十二的预热活动）。
2. 5:00-10:00和8:00-21:00是用户活跃度迅速增长的时段，21:00-22:00是一天中用户最活跃时段，且下单量也是最高的，应着重在这个时间段进行营销推广活动。
3. 整体用户来看，人均购买次数为2次，而购买过商品的用户人均购买次数3次，复购率高达66%，应考虑拓宽获客渠道，保证产品质量、价格优势及维护老客的同时，要增加新用户量。
4. 付费用户相较于非付费用户多贡献了35.84%的PV，所以要对这些活跃用户建立深度的用户画像，进行更精准的商品推荐。
5. 55%的用户在加入购物车后会购买商品，实行适当的满减活动，刺激消费者购物车内比价，采取限时优惠券增加用户购买的紧迫感，促成购买行为。47%的用户在收藏后会购买产品，建议在用户收藏后能有优惠、促销提示，促使用户下单。
6. 用户在加入购物车或者收藏产品后的一小时内购买商品的概率是最高的，所以建议在用户有前两种操作时立刻有客服跟进，提供优惠放送、话术吸引，帮助用户决策，刺激用户快速下单购买。
7. 整体的用户行为转化率只有2%，98%的行为是没有转化为购买的，用户浏览详情页后即流失。建议封面及标题改进，进一步吸引用户提升浏览量，对活动提醒、优惠券及产品详情页进行适当调整，应更加醒目让用户产生购买冲动，以提高转化率。
8. itemid为3122135的商品在9天内销售134单，在这份数据中是表现最好的，商家应该针对自己的热销产品建立粉丝群、微信群，实行社群管理，同时保证供应链畅通。
9. 大部分商品发生购买前累计用户行为在0-100次之间，要减少用户操作，提升客户购买效率及体验，例如对多次光顾店铺的顾客进行私信推荐、邀请进社群。
10. 根据RFM模型对用户分类后发现重要价值客户较少，用户类型主要分布在重要保持客户和重要发展客户。同样的，需要提高社群管理的效率，将重要保持客户和重要发展客户转变为重要价值客户。可采取定期发送文案、赠送优惠券等方式。要抓住用户渴望被认同、提升优越感、爱占便宜等心理进行考虑。